In [633]:
import pandas as pd
import numpy as np
import string
import pickle
import annoy

from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Продуктовые запросы берутся из "ProductsDataset.csv".  
Непродуктовые -- из "Болталки" 8-го юнита.

Загрузим необходимые данные

In [634]:
# !wget https://clck.ru/3724V4 -O ProductsDataset.csv

In [635]:
# Объединеним столбцы "title" и "descrirption". Также вставим столбец с обозначением продуктовых запросов [label=1]
df_1 = pd.read_csv("ProductsDataset.csv")
df_1["title_descrirption"] = df_1["title"] + " " + df_1["descrirption"]
df_1 = df_1.rename(columns={"title_descrirption": "text"})
df_1 = df_1[["title", "text","product_id"]]
df_1["labels"] = np.ones(len(df_1), dtype=int)
# Удалим пропуски в наших данных
df_1 = df_1.dropna()
df_1.sample(3)

,title,text,product_id,labels
225,Новые брюки на мальчика,Новые брюки на мальчика Новые льняные брюки ba...,5aaf758966fb077920118582,1
9267,"2 рубашки Henderson, Tatuum, Victor Alferi","2 рубашки Henderson, Tatuum, Victor Alferi 1) ...",5a59ed23d6775044877a2c5c,1
837,Наклейки stick'n click для фотосессии,Наклейки stick'n click для фотосессии Цена за ...,59cc92a7a380b65c7b654b58,1


In [636]:
# !wget https://clck.ru/3725rC -O QuestionsDataset.zip

In [637]:
# Считаем файл с данными для болталки в кол-ве 36000 строк. И также вставим столбец с обозначением не продуктовые запросы [label=0]
df_2 = pd.read_csv("QuestionsDataset.zip", nrows=36000,).rename(columns={"Вопрос": "text", "Ответ": "product_id"})
df_2["labels"] = np.zeros(len(df_2), dtype=int)
# Удалим пропуски в наших данных
df_2 = df_2.dropna()
df_2.sample(3)

,text,product_id,labels
2568,Остап Сулейман Берта-Мария Бендер-бей.Как он о...,Живет себе в Рио.... \n,0
28473,Выгоден ли миру атеизм? .,"Этот материальный мир - царство сатаны, а сата...",0
4743,"Девушки, а вам бы понравилось если на члене у ...",Я Люблю на члене рисовать и писать всякие прик...,0


Осуществим препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).

In [638]:
# В данной функции будет осуществляться препроцессинг текста
def preprocess_txt(line, morpher, sw, exclude):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [639]:
tqdm.pandas()

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [640]:
df_1['Preprocessed_texts'] = df_1.progress_apply(lambda row: preprocess_txt(row['text'], morpher, sw, exclude), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████| 33534/33534 [00:58<00:00, 569.80it/s]


In [641]:
df_1['title'] = df_1.progress_apply(lambda row: preprocess_txt(row['title'], morpher, sw, exclude), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████| 33534/33534 [00:07<00:00, 4650.24it/s]


In [642]:
df_2['Preprocessed_texts'] = df_2.progress_apply(lambda row: preprocess_txt(row['text'], morpher, sw, exclude), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████| 35999/35999 [01:05<00:00, 548.65it/s]


Перейдем к векторизации текста

In [645]:
# Приведем все предложения к одинаковой длине, задав её 100 
# Максимальная длина предложения
max_length = 100 # Укажите фиксированную длину
df_1['padded_sentences'] = df_1['Preprocessed_texts'].apply(lambda x: x + [0] * (max_length - len(x)) if len(x) < max_length else x[:max_length])
df_2['padded_sentences'] = df_2['Preprocessed_texts'].apply(lambda x: x + [0] * (max_length - len(x)) if len(x) < max_length else x[:max_length])

Для подготовки данных для обучения классификатара, объедини продуктовые запросы и болталки

In [646]:
# Объединим два датафрэйма в один
df = pd.concat([df_1, df_2], ignore_index=True)
# Удалим пропуски в наших данных
# df = df.dropna()

In [647]:
df.head(3)

,title,text,product_id,labels,Preprocessed_texts,padded_sentences
0,"[юбка, детский, orby]","Юбка детская ORBY Новая, не носили ни разу. В ...",58e3cfe6132ca50e053f5f82,1,"[юбка, детский, orby, новый, носить, реал, кра...","[юбка, детский, orby, новый, носить, реал, кра..."
1,[ботильон],"Ботильоны Новые,привезены из Чехии ,указан раз...",5667531b2b7f8d127d838c34,1,"[ботильон, новыепривезти, чехия, указать, разм...","[ботильон, новыепривезти, чехия, указать, разм..."
2,[брюки],Брюки Размер 40-42. Брюки почти новые - не зна...,59534826aaab284cba337e06,1,"[брюки, размер, 4042, брюки, новый, знать, мер...","[брюки, размер, 4042, брюки, новый, знать, мер..."


In [648]:
# Обучим модель Word2Vec на полученных данных
model =  Word2Vec(df['padded_sentences'], vector_size=100, window=5, min_count=1, workers=4)
model_prod = Word2Vec(df_1['title'], vector_size=100, window=5, min_count=1, workers=4)
model_mail = Word2Vec(df_2['padded_sentences'], vector_size=100, window=5, min_count=1, workers=4)

In [649]:
# Сохраненим векторизаванные модели
model.save("word2vec.model")
model_prod.save("word2vec_prod.model")
model_mail.save("word2vec_mail.model")

In [650]:
# Векторизация слов
def vectorize_sentence(sentence):
    vectorized_words = [model.wv[word] for word in sentence if word in model.wv.key_to_index]
    if len(vectorized_words) > 0:
        return np.mean(vectorized_words, axis=0).astype(np.float64)
    else:
        return np.zeros((100,), dtype=np.float64)

In [651]:
df['vectorized_sentences'] = df['padded_sentences'].progress_apply(vectorize_sentence)        

100%|█████████████████████████████████████████████████████████████████████████████████████| 69533/69533 [00:05<00:00, 11968.61it/s]


In [652]:
# Предсказания для векторизованных предложений
X = np.vstack(df['vectorized_sentences'].values)
y = np.array(df['labels']) # Метки классов

Разделем выборку на обучающую и валидационную.

In [653]:
# Разделение набора данных на обучающий и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [654]:
# Посмотрим на размер тренировочных данных
X_train.shape, y_train.shape

((55626, 100), (55626,))

In [655]:
# Посмотрим на размер тестовых данных
X_test.shape, y_test.shape

((13907, 100), (13907,))

Обучим классификатор с расчётом метрик на валидации

In [656]:
# Обучение модели логистической регрессии
lg_model = LogisticRegression(random_state=42, max_iter=500)
lg_model.fit(X_train, y_train)

# Оценка производительности модели
accuracy = lg_model.score(X_test, y_test)
print("LinearLR, Vectors: ", accuracy)

LinearLR, Vectors:  0.958078665420292


Сохраним обученную модель.

In [657]:
# Сохранение обученной модели
with open('trained_logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(lg_model, file)

#### Реализация поиска похожих товаров в контентной части бота

Этот код является определением функции `get_answer`. Он принимает три аргумента: `question` (строка), `model_lg` (объект LogisticRegression) со значением по умолчанию `model`, и `word2vec` (объект Word2Vec) со значением по умолчанию `Word2Vec`. Функция возвращает строку.

Таким образом, этот код определяет функцию `get_answer` с определенными типами аргументов и возвращаемого значения, и значениями по умолчанию для `model_lg` и `word2vec`.

* Все названия товаров свёрнем в векторное представление Word2Vec (на предобученном исходном датасете).
* Построен индекс по названиям документов.
* Для товарных запросов реализован поиск в индексе (запрос также оборачивается Word2Vec, происходит проход в индекс).

In [659]:
# Данная фунция сворачивает все продуктовые запросы в векторное представление
index_prod = annoy.AnnoyIndex(100 ,'angular')
df_prod = df_1[['title', 'product_id']]
index_map_products = {}
counter = 0

for index_, row in tqdm(df_prod.iterrows()): 
    n_w2v = 0
    index_map_products[counter] = row.iloc[1]
    question = row.iloc[0]
    vector = np.zeros(100)
    for word in question:
        if word in model_prod.wv:
            vector += model_prod.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index_prod.add_item(counter, vector)
            
    counter += 1
    
index_prod.build(10)
index_prod.save('products_.ann')

33534it [00:01, 22591.08it/s]


True

In [660]:
# Данная фунция сворачивает все запросы из болталки в векторное представление
index_mail = annoy.AnnoyIndex(100 ,'angular')
df_mail = df_2[['Preprocessed_texts', 'product_id']]
index_map_mail = {}
counter = 0

for index_, row in tqdm(df_mail.iterrows()): 
    n_w2v = 0
    index_map_mail[counter] = row.iloc[1]
    question = row.iloc[0]
    vector = np.zeros(100)
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index_mail.add_item(counter, vector)
            
    counter += 1

index_mail.build(10)
index_mail.save('mail_.ann')

35999it [00:02, 16227.20it/s]


True

In [665]:
# функция выдает на запрошенный предобработанный вопрос готовый ответ
def find_answer(question, model, index, index_map):
    # preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    if model == model_prod:
        answer_index = index.get_nns_by_vector(vector, 1, search_k=-2)
        # print('прод=',answer_index)
    else: 
        answer_index = index_mail.get_nns_by_vector(vector, 1, search_k=-1)     
        # print('не прод=',answer_index)
    return index_map[answer_index[0]] 

In [662]:
# Функция определяет вид запроса (продуктовый или нет) и запрашивает и функции find_answer ответ на преодработанные вопрос
def get_answer(question: str, model_lg: LogisticRegression = lg_model, word2vec: Word2Vec = Word2Vec) -> str:
    
    '''
    Определяет вид запроса (продуктовый или нет) и находит ответ,
    для продуктового запроса - product_id продукта, для непродутового - текст.
    '''
    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    preprocess_text = preprocess_txt(question, morpher, sw, exclude)
    vector = vectorize_sentence(preprocess_text)
    # print(vector)
    # Преобразуем список векторизованных предложений в массив NumPy
    # vector = np.stack(vectorize_question)

    if lg_model.predict(vector.reshape(1, -1))[0] == 1:
        # print("Продуковый запрос")
        return find_answer(preprocess_text, model_prod, index_prod, index_map_products)
    else:
        # print("Не продуктовый запрос")
        return find_answer(preprocess_text, model_mail, index_mail, index_map_mail)

In [663]:
# Загрузим обученной модель предсказания вопроса: продутовый или нет
with open('trained_logistic_regression_model.pkl', 'rb') as file:
    lg_model= pickle.load(file)

In [666]:
print("Ответ на продуктовый запрос, ",get_answer("Юбка детская ORBY"))
print("Ответ на болталку, ", get_answer("Где ключи от танка"))

Ответ на продуктовый запрос,  58e3cfe6132ca50e053f5f82
Ответ на болталку,  Рекомендации для всех-Тигр – символ года, животное серьезное, и четко определяет цветовую гамму новогодних нарядов – это его любимые цвета тигровых оттенков – оранжевые, золотистые, желтые. В одежде желательно иметь что-то полосатое. <br> <br>Не забудьте про украшения - аксессуары и украшения рекомендуется подобрать из натуральных материалов, камней. Хороши металлические кольца, бусы, браслеты, серьги из золота, серебра, меди и платины (но не более двух различных металлов) . Прекрасно подойдут бижутерия и украшения из ювелирных сплавов, например цепочки . Для одежды можно использовать украшения из кожи, меха, хлопка и др. Обратите внимание и на вашу обувь: металлический Тигр очень обидится, если вы будете встречать его 2010 год в обыкновенных домашних тапочках. Не бывать этому. Женщинам нужны кожаные туфли, мужчинам – ботинки, желательно с каким-нибудь броским декоративным элементом, например, с блестящей металл